In [1]:
import json
import redis
import paho.mqtt.client as mqtt 

#connect to a broker to send/receive messages to/from the broker
client = mqtt.Client()

seriesName = "emergencySiren"

REDIS_HOST = "redis-17637.c114.us-east-1-4.ec2.cloud.redislabs.com"
REDIS_PORT = 17637
REDIS_USER = "default"
REDIS_PASSWORD = "fFh38GR6BxHI0xVoGbsUKAzPZlCBXLAA"

redis_client = redis.Redis(host = REDIS_HOST, port = REDIS_PORT, password = REDIS_PASSWORD, username= REDIS_USER)
print("Is connected:", redis_client.ping())


Is connected: True


In [2]:
def check_series_creation(redis_client, sensor):
    '''checks for the existence of the series and creates them'''
    try:
        redis_client.ts().create(sensor+f":{seriesName}", label = {f"{seriesName}"})
    except redis.ResponseError:
        pass
    

def on_connect(client, userdata, flags, rc):
    #subscribe to a topic
    client.subscribe('s291871')


def on_message(client, userdata, msg):
    message = msg.payload.decode() 
    message_dict = json.loads(message)
    print(message)
    #whenever a new sensor is received, we need to create the redis timeseries
    check_series_creation(redis_client, message_dict["sensor"])
    
    #store the data
    redis_client.ts().add(message_dict["sensor"]+f":{seriesName}", message_dict["timestamp"], message_dict["emergencysiren"])
    


#bind the on_connect and on_message method to the client
client.on_connect = on_connect
client.on_message = on_message

#connect to the broker 
client.connect('mqtt.eclipseprojects.io', 1883)

0

In [3]:
#keep listening over the topic we subscribed to using an infinite loop 
client.loop_forever()

{"sensor": "PortaNuova", "timestamp": 1681897712892, "emergencysiren": 1}
{"sensor": "PortaNuova", "timestamp": 1681897718958, "emergencysiren": 1}
{"sensor": "PortaNuova", "timestamp": 1681897724825, "emergencysiren": 1}
{"sensor": "PortaNuova", "timestamp": 1681897726982, "emergencysiren": 1}
{"sensor": "PortaNuova", "timestamp": 1681897730506, "emergencysiren": 1}
{"sensor": "PortaNuova", "timestamp": 1681897733711, "emergencysiren": 1}
{"sensor": "PortaNuova", "timestamp": 1681897736667, "emergencysiren": 1}
{"sensor": "PortaNuova", "timestamp": 1681897739497, "emergencysiren": 1}
{"sensor": "PortaNuova", "timestamp": 1681897742704, "emergencysiren": 1}
{"sensor": "PortaNuova", "timestamp": 1681897744372, "emergencysiren": 1}
{"sensor": "PortaSusa", "timestamp": 1681897765858, "emergencysiren": 1}
{"sensor": "PortaSusa", "timestamp": 1681897768762, "emergencysiren": 1}
{"sensor": "PortaSusa", "timestamp": 1681897771997, "emergencysiren": 1}
{"sensor": "PortaSusa", "timestamp": 1681

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=957cfe07-d816-4770-9cc0-d7c1b4767886' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>